<a href="https://colab.research.google.com/github/iraidaantropova/StrD/blob/main/HW2_SinitsaI_StrD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=296e505834b101b366d84f74d51acad8d077e9dae5ba5aa8d1db115d96ff2335
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.5.0/kafka_2.13-3.5.0.tgz

!tar -xzf kafka_2.13-3.5.0.tgz

In [3]:
!./kafka_2.13-3.5.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.5.0/config/zookeeper.properties
!./kafka_2.13-3.5.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.5.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10

Waiting for 10 secs until kafka and zookeeper services are up and running


In [4]:
!ps -ef | grep kafka

root        1617       1  2 02:59 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.13-3.5.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.13-3.5.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.13-3.5.0/bin/../config/log4j.properties -cp /content/kafka_2.13-3.5.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.13-3.5.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.13-3.5.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.13-3.5.0/bin/../libs/audience-annotations-0.13.0.jar:/content/kafka_2.13-3.5.0/bin/../libs/commons-cli-1.4.jar:/content/kafka_2.13-3.5.0/bin/../libs/commons-lang3-3.8.1.jar:/content/kafka_2.1

In [5]:
#создаем топик( таблица в кафке с параметрами партиционирования, репликации и retention time) с параметрами: partitions=2, replication-factor=1
!./kafka_2.13-3.5.0/bin/kafka-topics.sh

Create, delete, describe, or change a topic.
Option                                   Description                            
------                                   -----------                            
--alter                                  Alter the number of partitions and     
                                           replica assignment. Update the       
                                           configuration of an existing topic   
                                           via --alter is no longer supported   
                                           here (the kafka-configs CLI supports 
                                           altering topic configs with a --     
                                           bootstrap-server option).            
--at-min-isr-partitions                  if set when describing topics, only    
                                           show partitions whose isr count is   
                                           equal to the configur

In [6]:
# Создаем топик
!./kafka_2.13-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic testtopic --config retention.ms=-1

Created topic testtopic.


In [7]:
# Проверяю наличие топика
!./kafka_2.13-3.5.0/bin/kafka-topics.sh  --list --bootstrap-server localhost:9092

testtopic


In [8]:
#проверяю

!./kafka_2.13-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic testtopic

Topic: testtopic	TopicId: SJl8b1eDSb6SCP3_lBQNxQ	PartitionCount: 2	ReplicationFactor: 1	Configs: retention.ms=-1
	Topic: testtopic	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: testtopic	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [9]:
#создаю продюсера (сервер, который поставляет сообщения в топик), отправляю несколько сообщений в топик
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testtopic

>1
>2
>3
>

In [10]:
# Добавляю в топик новые данные
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testtopic

>7
>8
>9
>

In [11]:
# Читаю записанный топик
!./kafka_2.13-3.5.0/bin/kafka-console-consumer.sh --topic testtopic --from-beginning --bootstrap-server localhost:9092

7
8
9
1
2
3
Processed a total of 6 messages


In [19]:
# Удаляю топик
!./kafka_2.13-3.5.0/bin/kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --delete --topic testtopic

In [20]:
# Проверка
!./kafka_2.13-3.5.0/bin/kafka-topics.sh  --list --bootstrap-server localhost:9092

__consumer_offsets


In [21]:
# Создаю топик со временем жизни 1 минута
!./kafka_2.13-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 2 --topic testtopic --config retention.ms=60000

Created topic testtopic.


In [22]:
# Проверка

!./kafka_2.13-3.5.0/bin/kafka-topics.sh  --list --bootstrap-server localhost:9092

__consumer_offsets
testtopic


In [23]:
# Описание
!./kafka_2.13-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic testtopic

Topic: testtopic	TopicId: HB0v0TPPS-SonvC48Ia6jA	PartitionCount: 2	ReplicationFactor: 1	Configs: retention.ms=60000
	Topic: testtopic	Partition: 0	Leader: 0	Replicas: 0	Isr: 0
	Topic: testtopic	Partition: 1	Leader: 0	Replicas: 0	Isr: 0


In [24]:
# Создаю продюсера и отправляю несколько сообщений в топик
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testtopic

>5
>6
>7
>

In [25]:
# Дописываю
!./kafka_2.13-3.5.0/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic testtopic

>12
>13
>14
>

In [26]:
# Создаю консьюмера и читаю отправленные сообщения
!./kafka_2.13-3.5.0/bin/kafka-console-consumer.sh --topic testtopic --from-beginning --bootstrap-server localhost:9092

12
13
14
Processed a total of 3 messages


In [27]:
!spark-submit --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.20.1
Branch HEAD
Compiled by user ubuntu on 2023-09-09T01:53:20Z
Revision ce5ddad990373636e94071e7cef2f31021add07b
Url https://github.com/apache/spark
Type --help for more information.


In [28]:
!pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.apache.kafka:kafka-clients:3.5.0

Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7921219f-a1b0-40c9-85cc-c5837bb7c3e6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
